In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd /content/drive/MyDrive/car-classification-api/car-classification-api


/content/drive/MyDrive/car-classification-api/car-classification-api


In [ ]:
!git status

In [2]:
# !pip install torch torchvision matplotlib

In [3]:
from PIL import Image
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import random_split, DataLoader, Subset
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import pandas as pd

In [17]:
# Define augmentations to be carried out
train_transformers = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    # transforms.RandomErasing(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transformers = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [18]:
# Load the train dataset
train_data = datasets.ImageFolder("/content/drive/MyDrive/car-classification-api/data/train")

# Split the data further into train and val sets
train_size = int(0.8*len(train_data))
val_size = len(train_data) - train_size
train_indices, val_indices = random_split(range(len(train_data)), [train_size, val_size])
# train_dataset, val_dataset = random_split(train_data, [train_size, val_size])

In [19]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, original_dataset, indices, transform=None):
        self.original_dataset = Subset(original_dataset, indices)
        self.transform = transform

    def __len__(self):
        return len(self.original_dataset)

    def __getitem__(self, idx):
        img, label = self.original_dataset[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

In [20]:
# Define train and validation datasets with separate transforms
train_dataset = CustomDataset(train_data, train_indices, transform=train_transformers)
val_dataset = CustomDataset(train_data, val_indices, transform=val_transformers)

In [21]:
img, label = train_dataset[3]  # Replace 0 with any valid index
print(type(img))  # Should show <class 'torch.Tensor'>
print(img.shape)  # Should show [3, 224, 224] or similar

<class 'torch.Tensor'>
torch.Size([3, 224, 224])


In [22]:
for i in range(15):  # Inspect the first 5 samples
  img, label = train_dataset[i]
  print(f"Sample {i}: Type - {type(img)}, Shape - {img.shape}")

Sample 0: Type - <class 'torch.Tensor'>, Shape - torch.Size([3, 224, 224])
Sample 1: Type - <class 'torch.Tensor'>, Shape - torch.Size([3, 224, 224])
Sample 2: Type - <class 'torch.Tensor'>, Shape - torch.Size([3, 224, 224])
Sample 3: Type - <class 'torch.Tensor'>, Shape - torch.Size([3, 224, 224])
Sample 4: Type - <class 'torch.Tensor'>, Shape - torch.Size([3, 224, 224])
Sample 5: Type - <class 'torch.Tensor'>, Shape - torch.Size([3, 224, 224])
Sample 6: Type - <class 'torch.Tensor'>, Shape - torch.Size([3, 224, 224])
Sample 7: Type - <class 'torch.Tensor'>, Shape - torch.Size([3, 224, 224])
Sample 8: Type - <class 'torch.Tensor'>, Shape - torch.Size([3, 224, 224])
Sample 9: Type - <class 'torch.Tensor'>, Shape - torch.Size([3, 224, 224])
Sample 10: Type - <class 'torch.Tensor'>, Shape - torch.Size([3, 224, 224])
Sample 11: Type - <class 'torch.Tensor'>, Shape - torch.Size([3, 224, 224])
Sample 12: Type - <class 'torch.Tensor'>, Shape - torch.Size([3, 224, 224])
Sample 13: Type - <cla

In [23]:
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [24]:
for inputs, labels in train_loader:
    print(type(inputs), inputs.shape)  # Should work without error
    break

<class 'torch.Tensor'> torch.Size([32, 3, 224, 224])


In [25]:
inputs, labels = next(iter(train_loader))
print(inputs.shape, labels.shape)

torch.Size([32, 3, 224, 224]) torch.Size([32])


In [26]:
# Load pre-trained model, ResNet
model = models.resnet50(pretrained=True)

# Modify final layer
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, len(train_data.classes))

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [28]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [29]:
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10

train_losses, val_losses = [], []
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()
        # Forward
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        # Backward
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_losses.append(running_loss / len(train_loader))

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

Epoch 1/10, Loss: 5.3491
Epoch 2/10, Loss: 5.2496
Epoch 3/10, Loss: 5.2057
Epoch 4/10, Loss: 5.1706


In [ ]:
y_true, y_pred = []

model.eval()
correct = 0
total = 0
val_loss = 0.0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())
        loss = criterion(outputs, labels)
        val_loss += loss.item()

val_losses.append(val_loss / len(val_loader))
print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}")

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Convert to DataFrame for better readability
cm_df = pd.DataFrame(cm, index=train_data.classes, columns=train_data.classes)
print(cm_df)

print(f"Validation Accuracy: {100 * correct / total:.2f}%")

In [ ]:
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()